In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from matplotlib import pyplot as plt

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sohamdatta05","key":"8b61fd65debb54177f471b54e9f298a7"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shuvoalok/raf-db-dataset

Dataset URL: https://www.kaggle.com/datasets/shuvoalok/raf-db-dataset
License(s): other


In [ ]:
!unzip -q raf-db-dataset.zip -d rafdb_dataset

In [ ]:
import os

In [ ]:
base_dir = '/content/rafdb_dataset'

In [ ]:
dataset_path = '/content/rafdb_dataset/DATASET'
print("Dataset contents:", os.listdir(dataset_path))

Dataset contents: ['train', 'test']


In [ ]:
import pandas as pd

train_labels_df = pd.read_csv(os.path.join(base_dir, 'train_labels.csv'))
print("Train labels sample:\n", train_labels_df.head())

Train labels sample:
                      image  label
0  train_00001_aligned.jpg      5
1  train_00002_aligned.jpg      5
2  train_00003_aligned.jpg      4
3  train_00004_aligned.jpg      4
4  train_00005_aligned.jpg      5


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = os.path.join(dataset_path, 'train')
print("Train directory contents:", os.listdir(train_dir))

Train directory contents: ['5', '1', '3', '4', '7', '2', '6']


In [ ]:
def convert_to_rgb(img):
    if len(img.shape) == 2:  # Grayscale image (H, W)
        img = np.stack([img] * 3, axis=-1)  # Convert to (H, W, 3)
    return img

In [ ]:
val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Match the split defined in train_datagen
    preprocessing_function=convert_to_rgb
)

In [ ]:
train_datagen = ImageDataGenerator(
                rescale=1./255,              # Normalize pixel values to [0, 1]
                validation_split=0.2,        # 20% of training data for validation
                rotation_range=20,           # Randomly rotate images by up to 20 degrees
                width_shift_range=0.2,       # Shift images horizontally by 20% of width
                height_shift_range=0.2,      # Shift images vertically by 20% of height
                horizontal_flip=True,        # Randomly flip images horizontally
                fill_mode='nearest',          # Fill in new pixels after rotation/shift
                preprocessing_function=convert_to_rgb
)




In [ ]:
train_generator = val_test_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(192, 192),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 9819 images belonging to 7 classes.


In [ ]:
validation_generator = val_test_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(192, 192),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2452 images belonging to 7 classes.


In [ ]:
test_dir = os.path.join(dataset_path, 'test')

In [ ]:
test_labels_df = pd.read_csv(os.path.join(base_dir, 'test_labels.csv'))
print("Test labels sample:\n", test_labels_df.head())

Test labels sample:
                    image  label
0  test_0001_aligned.jpg      5
1  test_0002_aligned.jpg      1
2  test_0003_aligned.jpg      4
3  test_0004_aligned.jpg      1
4  test_0005_aligned.jpg      5


In [ ]:
test_generator = val_test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(192, 192),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Set to False for test data to preserve order for evaluation
)

Found 3068 images belonging to 7 classes.


In [ ]:
print("Train samples:", train_generator.samples)
print("Validation samples:", validation_generator.samples)
print("Test samples:", test_generator.samples)
print("Class indices:", test_generator.class_indices)

Train samples: 9819
Validation samples: 2452
Test samples: 3068
Class indices: {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6}


In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D

CNN Model 1

In [ ]:
model = Sequential([

    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),


    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.50),


    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.50),


    layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    layers.BatchNormalization(),

    # Dense layers
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),

    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(train_generator, epochs=20, validation_data=validation_generator)

Epoch 1/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 113s 297ms/step - accuracy: 0.4011 - loss: 2.1234 - val_accuracy: 0.4955 - val_loss: 1.9098
Epoch 2/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 66s 215ms/step - accuracy: 0.6174 - loss: 1.1380 - val_accuracy: 0.6415 - val_loss: 1.1119
Epoch 3/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 66s 215ms/step - accuracy: 0.7178 - loss: 0.7917 - val_accuracy: 0.6639 - val_loss: 1.0057
Epoch 4/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 67s 219ms/step - accuracy: 0.7581 - loss: 0.6867 - val_accuracy: 0.7325 - val_loss: 0.7959
Epoch 5/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 66s 215ms/step - accuracy: 0.8308 - loss: 0.4729 - val_accuracy: 0.5616 - val_loss: 1.5753
Epoch 6/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 66s 214ms/step - accuracy: 0.8877 - loss: 0.3317 - val_accuracy: 0.5118 - val_loss: 1.3395
Epoch 7/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 66s 215ms/step - accuracy: 0.9123 - loss: 0.2613 - val_accuracy: 0.7157 - val_loss: 1.0143
Epoch 8/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 66s 213ms/step - accuracy: 0.9493 - loss: 

In [ ]:
cnn_loss, cnn_acc = model.evaluate(test_generator)
print(f"Simple CNN Test Accuracy: {cnn_acc:.4f}")

96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - accuracy: 0.6747 - loss: 1.6800
Simple CNN Test Accuracy: 0.7194


In [ ]:
model.save('cnn1.h5')

In [ ]:
from tensorflow.keras.applications import ResNet50

# Load ResNet50 base
convolutional_base = ResNet50(weights='imagenet', include_top=False, input_shape=(192, 192, 3))

# Define model
resnet_model = Sequential()
resnet_model.add(layers.Input(shape=(192, 192, 3)))
resnet_model.add(convolutional_base)
resnet_model.add(layers.Flatten())
resnet_model.add(layers.BatchNormalization())
resnet_model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
resnet_model.add(layers.Dropout(0.3))
resnet_model.add(layers.BatchNormalization())
resnet_model.add(layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
resnet_model.add(layers.Dropout(0.3))
resnet_model.add(layers.BatchNormalization())
resnet_model.add(layers.Dense(7, activation='softmax'))

resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
resnet_model.fit(train_generator, epochs=20, validation_data=validation_generator)

Epoch 1/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 76s 246ms/step - accuracy: 0.6953 - loss: 1.7566 - val_accuracy: 0.6166 - val_loss: 1.8620
Epoch 2/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 76s 248ms/step - accuracy: 0.7391 - loss: 1.4757 - val_accuracy: 0.6709 - val_loss: 1.6050
Epoch 3/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 75s 245ms/step - accuracy: 0.7602 - loss: 1.3665 - val_accuracy: 0.7215 - val_loss: 1.5448
Epoch 4/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 76s 246ms/step - accuracy: 0.7699 - loss: 1.2958 - val_accuracy: 0.6521 - val_loss: 1.5925
Epoch 5/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 76s 247ms/step - accuracy: 0.7780 - loss: 1.2734 - val_accuracy: 0.6937 - val_loss: 1.6015
Epoch 6/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 76s 247ms/step - accuracy: 0.8080 - loss: 1.2152 - val_accuracy: 0.7153 - val_loss: 1.4963
Epoch 7/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 76s 246ms/step - accuracy: 0.8203 - loss: 1.1127 - val_accuracy: 0.7263 - val_loss: 1.4706
Epoch 8/20
307/307 ━━━━━━━━━━━━━━━━━━━━ 76s 247ms/step - accuracy: 0.8383 - loss: 1

In [ ]:
cnn_loss, resnet_acc = resnet_model.evaluate(test_generator)
print(f"Simple CNN Test Accuracy: {resnet_acc:.4f}")

96/96 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.7406 - loss: 1.6005
Simple CNN Test Accuracy: 0.7523


In [ ]:
resnet_model.save('resnet2.h5')

In [ ]:
#fix overfitting of resnet model